In [3]:
# Import selenium and its dependencies
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, TimeoutException, StaleElementReferenceException

# import pandas 
import pandas as pd
import time
base_url = "https://members.ocpinfo.com/tcpr/public/pr/en/#/forms/new/?table=0x800000000000003D&form=0x800000000000002C&command=0x80000000000007C5&donotreload=ShowSRL"

scrape_df = pd.DataFrame()
scrape_df["Number of Corporations"] = 0


In [6]:
# launch the browser driver
driver = webdriver.Firefox()


In [12]:
scrape_df = pd.DataFrame()
scrape_df["Number of Corporations"] = 0

# Loop through each page
for i in range(367, 2822):
    
    print(f"Page: {i}")
    
    # go the page url and wait 2 seconds for the page to load
    driver.get(f"{base_url}{i}")
    time.sleep(3)
    
    # check if the element is present
    try:
        element_present = EC.presence_of_element_located((By.ID, 'SearchReasultList'))
        WebDriverWait(driver, 5).until(element_present)
    except TimeoutException:
        print("Timed out waiting for page to load")
        
    # get the search results div
    search_divs = driver.find_element(By.ID, "SearchReasultList").find_elements(By.CLASS_NAME, "f15") 

    # loop through each entry in the div table
    for div in search_divs:
        
        # get the name and href of each pharmacist
        name = div.find_element(By.TAG_NAME, "a").text
        href =  div.find_element(By.TAG_NAME, "a").get_attribute("href")
        data = {
            "name": name,
            "href": href,
        }

        # append the data to the dataframe
        scrape_df = pd.concat([scrape_df, pd.DataFrame([data])], ignore_index=True)
    


Page: 367
Page: 368
Page: 369
Page: 370
Page: 371
Page: 372
Page: 373
Page: 374
Page: 375
Page: 376
Page: 377
Page: 378
Page: 379
Page: 380
Page: 381
Page: 382
Page: 383
Page: 384
Page: 385
Page: 386
Page: 387
Page: 388
Page: 389
Page: 390
Page: 391
Page: 392
Page: 393
Page: 394
Page: 395
Page: 396
Page: 397
Page: 398
Page: 399
Page: 400
Page: 401
Page: 402
Page: 403
Page: 404
Page: 405
Page: 406
Page: 407
Page: 408
Page: 409
Page: 410
Page: 411
Page: 412
Page: 413
Page: 414
Page: 415
Page: 416
Page: 417
Page: 418
Page: 419
Page: 420
Page: 421
Page: 422
Page: 423
Page: 424
Page: 425
Page: 426
Page: 427
Page: 428
Page: 429
Page: 430
Page: 431
Page: 432
Page: 433
Page: 434
Page: 435
Page: 436
Page: 437
Page: 438
Page: 439
Page: 440
Page: 441
Page: 442
Page: 443
Page: 444
Page: 445
Page: 446
Page: 447
Page: 448
Page: 449
Page: 450
Page: 451
Page: 452
Page: 453
Page: 454
Page: 455
Page: 456
Page: 457
Page: 458
Page: 459
Page: 460
Page: 461
Page: 462
Page: 463
Page: 464
Page: 465
Page: 466


In [14]:
# for each entry in the dataframe go to their profile url
for index, row in scrape_df.iterrows():
    # go to there profile url
    driver.get(row["href"])
    time.sleep(2.25)
    name = scrape_df.at[index, "name"]
    # not all entries have a number of corporations so try and get it
    # other wise set it to 0
    try:
        num_corp = len(driver.find_element(By.ID, "OtherCorpoContent").find_elements(By.TAG_NAME, "div"))
        print(f"{name} has {num_corp} corporations")
        scrape_df.at[index, "Number of Corporations"] = num_corp
    except NoSuchElementException:
        num_corp = 0
        print(f"{name} has {num_corp} corporations")
        scrape_df.at[index, "Number of Corporations"] = num_corp

C:\Users\shann\AppData\Local\Temp\ipykernel_27884\2104750938.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  scrape_df.iloc[index]["Number of Corporations"] = len(driver.find_element(By.ID, "OtherCorpoContent").find_elements(By.TAG_NAME, "div"))


TimeoutException: Message: Navigation timed out after 300000 ms
Stacktrace:
RemoteError@chrome://remote/content/shared/RemoteError.sys.mjs:8:8
WebDriverError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:188:5
TimeoutError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:675:5
bail@chrome://remote/content/marionette/sync.sys.mjs:211:19


In [13]:
scrape_df.to_csv("ocp_pharmacist_scrape.csv" , index=False)

In [11]:
scrape_df = pd.read_csv("ocp_pharmacist_scrape.csv")
scrape_df

,Number of Corporations,name,href
0,NaN,Brenda Kukura,https://members.ocpinfo.com/tcpr/public/pr/en/...
1,NaN,Brenda Lawson,https://members.ocpinfo.com/tcpr/public/pr/en/...
2,NaN,Brenda Lorenz,https://members.ocpinfo.com/tcpr/public/pr/en/...
3,NaN,Brenda Morden,https://members.ocpinfo.com/tcpr/public/pr/en/...
4,NaN,Brenda Oh,https://members.ocpinfo.com/tcpr/public/pr/en/...
...,...,...,...
24538,NaN,Zuoqian (Bambi) Zhang,https://members.ocpinfo.com/tcpr/public/pr/en/...
24539,NaN,Zuzana Simkova,https://members.ocpinfo.com/tcpr/public/pr/en/...
24540,NaN,Zuzanna Szmukier,https://members.ocpinfo.com/tcpr/public/pr/en/...
24541,NaN,Zvenyslava Dovhanyk,https://members.ocpinfo.com/tcpr/public/pr/en/...
